In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

*解压数据集*

In [ ]:
!unzip ./data/data210802/训练集.zip > /dev/null
!unzip ./data/data210802/测试集a.zip > /dev/null


replace 训练集/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
replace 测试集a/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [24]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder


#得到训练集的商品属性csv文件
train_goods = pd.concat([
    pd.read_csv('./训练集/traindata_goodsid/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00002', header=None, names=['goods_id', 'cat_id', 'brandsn'])], axis=0)

# train_goods.head(5)


In [25]:
#得到训练集的用户行为csv文件
train_user = pd.concat([pd.read_csv(x, header=None, names=['user_id', 'goods_id', 'is_clk', 'is_like', 'is_addcart', 'is_order', 'expose_start_time', 'dt'], nrows=None)
for x in glob.glob('./训练集/traindata_user/part*')], axis=0)

# train_user.head(5)

In [26]:
#得到测试集的商品属性csv文件
test_goods = pd.concat([
    pd.read_csv('./测试集a/predict_goods_id/part-00000', header=None, names=['goods_id']),
    pd.read_csv('./测试集a/predict_goods_id/part-00001', header=None, names=['goods_id']),
], axis=0)
# test_goods.head(1)

In [27]:
#得到测试集的用户id
test_user = pd.read_excel('./测试集a/a榜需要预测的uid_5000.xlsx')
test_user.head(5)

,user_id
0,db55512e13c666a67dbfeae200bbb77e
1,6be82d1677c1eacfc615446329fa90b1
2,04672f37371f47e98247e3a426e33d44
3,887343b7ef8d77baed6b2fdfc4b8f536
4,95b336152a9e6aa6130233846ddcfc7a


In [28]:
import math
train_user['date'] = train_user['dt'].apply(lambda x: math.exp((30-int(str(x)[6:]))/15*math.log(0.5)) if x else 0 ).round(2)
train_user = train_user[train_user['user_id'].isin(test_user['user_id'])&train_user['goods_id'].isin(test_goods['goods_id'])]
train_user.eval('score = (is_clk+is_like*2+is_addcart*2.5+is_order*3.5)*date' , inplace=True)






In [29]:
postive_user =  train_user.groupby('user_id').agg({
    'is_order': 'sum'
    })
postive_user =postive_user[postive_user['is_order']>0]
postive_user = postive_user.reset_index()
postive_user.columns=[
    'user_id',
    'is_order_sum',
]


In [30]:
train_user = train_user[train_user['is_order'].isin(postive_user['is_order_sum'])]
train_user = train_user.drop(columns=['is_clk','is_addcart','is_like','is_order','expose_start_time', 'dt','date'])

test_user = test_user[test_user['user_id'].isin(train_user['user_id'])]
test_goods = test_goods[test_goods['goods_id'].isin(train_goods['goods_id'])]

#对用户编号和商品编号进行数字编码
user_encode = LabelEncoder()
user_encode.fit(list(train_user['user_id']))

goods_encode = LabelEncoder()
goods_encode.fit(list(train_user['goods_id'])+list(train_goods['goods_id']))

LabelEncoder()

In [31]:
test_user.shape

(2302, 1)

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
train_user['goods_id'] = goods_encode.transform(train_user['goods_id'])
train_user['user_id'] = user_encode.transform(train_user['user_id'])
test_user['user_id'] =  user_encode.transform(test_user['user_id'])
test_goods['goods_id'] = goods_encode.transform(test_goods['goods_id'])
hasBuy = {}
t = train_user.groupby(['user_id'])
for e in t:
    k = e[0]
    hasBuy[k] = list(e[1]['goods_id'].values)

# print(t_user.describe())
# print('*'*100)
# print(train_user.describe())
# sns.countplot(x='score',data=train_user,hue=None)
# plt.show()
# train_user['score'].hist()
# plt.show()
# train_user.head(5)
# train_user['user_id'].nunique(), train_user['goods_id'].nunique()

In [33]:
import math
import random


class BasicSVD:
    def __init__(self, R, m, n, F, alpha, lambda1, lambdal2, max_iter=1):
        self.R = R
        self.m = m
        self.n = n
        self.F = F
        self.P = np.zeros((m,F))
        self.Q = np.zeros((n,F))
        self.alpha = alpha
        self.lambda1 = lambda1
        self.lambda2 = lambdal2
        self.max_iter = max_iter
        
    def fit(self):
        for i in range(self.m):
            self.P[i] = [random.random() / math.sqrt(self.F) for x in range(0, self.F)]
        for i in range(self.n):
            self.Q[i] = [random.random() / math.sqrt(self.F) for x in range(0, self.F)]
        for i in range(self.max_iter):
            print(i)
            for u_id, g_id,score in zip(self.R['user_id'].values, self.R['goods_id'].values,self.R['score'].values):
                    loss = score - np.dot(self.P[u_id], self.Q[g_id])
                    self.P[u_id] = self.P[u_id] + self.alpha * (
                            loss * self.Q[g_id] - self.lambda1 * self.P[u_id])
                    self.Q[g_id] = self.Q[g_id] + self.alpha * (
                            loss * self.P[u_id] - self.lambda2 * self.Q[g_id])

    def predict(self, user_id, goods_id):
        l1 = self.P[user_id]
        l2 = self.Q[goods_id]
        return np.dot(l1, l2)


In [34]:
model = BasicSVD(R=train_user, m=len(user_encode.classes_), n=len(goods_encode.classes_), F=100, alpha=0.003, lambda1=0.01, 
lambdal2=0.01, max_iter=500)
model.fit()
data = {'user_id':[],'goods_id':[]}

for x1 in test_user['user_id']:
    res = {}
    for y1 in test_goods['goods_id']:
        if hasBuy.get(x1) and y1 in hasBuy[x1]:
            continue
        r = model.predict(x1,y1)
        res[y1] = r
    l = sorted(res.items(), key=lambda x :x[1], reverse=True)
    data['user_id'].append(x1)
    data['goods_id'].append(l[0][0])
    data['user_id'].append(x1)
    data['goods_id'].append(l[1][0])
    data['user_id'].append(x1)
    data['goods_id'].append(l[2][0])
data['user_id'] = user_encode.inverse_transform(data['user_id'])
data['goods_id'] = goods_encode.inverse_transform(data['goods_id'])
df = pd.DataFrame(data)
df.to_csv('u2i.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 